In [12]:
import torch
from torch import nn #nn contains PyTorch building blocks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

#check torch version
torch.__version__

'1.13.1'

In [14]:
# CSV path
folder_name = "data"
file_name = "hungary_chickenpox.csv"
path = os.path.join('../',folder_name, file_name)
#Load CSV into Dataframe
df = pd.read_csv(path,sep=',')
df = df.drop(columns = ['Date'])
df.shape

(522, 20)

In [17]:
df.head(3)

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
1,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
2,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44


In [4]:
labels = np.arange(0,523,1)

In [23]:
embeddings = nn.Embedding(522,10)
embeddings.weight.requires_grad = False

In [21]:
class Generator(nn.Module):
    def __init__(self, embeddings,output_shape=3, input_dim=1, hidden_units=64):
        super(Generator, self).__init__()
        
        self.embeddings = embeddings
        self.linear = nn.Linear(200,100)
        
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(input_dim, hidden_units, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_units * 8),
            nn.ReLU(True),
            # state size.  hidden_units*8) x 4 x 4
            nn.ConvTranspose2d(hidden_units, hidden_units, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_units * 4),
            nn.ReLU(True),
            # state size.  hidden_units*4) x 8 x 8
            nn.ConvTranspose2d(hidden_units, hidden_units, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_units * 2),
            nn.ReLU(True),
            # state size.  hidden_units*2) x 16 x 16
            nn.ConvTranspose2d(hidden_units, hidden_units, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(True),
            # state size.  hidden_units) x 32 x 32
            nn.ConvTranspose2d(hidden_units,output_shape, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
        
        def forward(self, x, label_embed):
            label_embed = self.embeddings(label_embed)

            x = x.view(-1,100)
            x = torch.cat([x,label_embed], dim=1)

            x = self.linear(x)
            x = x.unsqueeze(2).unsqueeze(3)

            output = self.main(x)
            return output

In [22]:
gen

Generator(
  (l1): Linear(in_features=10, out_features=10, bias=True)
  (label_embedding): Embedding(522, 1)
  (l2): Linear(in_features=10, out_features=10, bias=True)
)